In [10]:
import numpy as np
import random
import time
from IPython.display import clear_output

class TownBoard:
    """Creates a town board with houses and trees (no roads)"""
    def __init__(self, size=15):
        self.size = size
        self.grid = np.zeros((size, size), dtype=int)
        self.cell_types = {
            0: {'name': 'tree', 'emoji': '🌲', 'flammable': True},
            1: {'name': 'house', 'emoji': '🏠', 'flammable': True},
            2: {'name': 'fire', 'emoji': '🔥', 'flammable': False},
            3: {'name': 'burned', 'emoji': '🪵', 'flammable': False}
        }
        
    def generate_town(self, house_coverage=0.3):
        """
        Generate random town layout with:
        - Houses covering 20-40% of the board in clusters
        - Rest filled with trees
        """
        # Fill entire board with trees initially
        self.grid.fill(0)
        
        # Calculate number of houses to place (20-40% coverage)
        total_cells = self.size * self.size
        target_houses = int(total_cells * random.uniform(0.2, 0.4))
        
        # Place houses in random clusters
        house_count = 0
        while house_count < target_houses:
            # Start new cluster
            cluster_size = random.randint(2, 5)
            start_x = random.randint(0, self.size-1)
            start_y = random.randint(0, self.size-1)
            
            # Place cluster
            for _ in range(cluster_size):
                if house_count >= target_houses:
                    break
                
                # Random walk from starting point
                x = min(max(0, start_x + random.randint(-1, 1)), self.size-1)
                y = min(max(0, start_y + random.randint(-1, 1)), self.size-1)
                
                if self.grid[y][x] == 0:  # Only replace trees
                    self.grid[y][x] = 1
                    house_count += 1
    
    def display(self):
        """Show the town with emojis"""
        clear_output(wait=True)
        for row in self.grid:
            print(' '.join([self.cell_types[cell]['emoji'] for cell in row]))
        print("\nLegend: 🌲 Tree | 🏠 House | 🔥 Fire | 🪵 Burned")



In [15]:
class FireSimulator:
    """Simulates fire spread through the town"""
    def __init__(self, town_board):
        self.board = town_board
        self.time_step = 0
        self.burning_cells = set()
    
    def ignite_random(self, num_fires=3):
        """Start random fires in the town"""
        for _ in range(num_fires):
            while True:
                x, y = random.randint(0, self.board.size-1), random.randint(0, self.board.size-1)
                if self.board.grid[y][x] in [0, 1]:  # Can ignite trees or houses
                    self._ignite_cell(x, y)
                    break
    
    def _ignite_cell(self, x, y):
        """Set a cell on fire"""
        self.board.grid[y][x] = 2
        self.burning_cells.add((x, y))
    
    def spread_fire(self):
        """Spread fire to adjacent cells"""
        new_fires = set()
        
        for (x, y) in self.burning_cells:
            # Check all 8 neighboring cells
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    if dx == 0 and dy == 0:
                        continue  # Skip current cell
                    
                    nx, ny = x + dx, y + dy
                    if (0 <= nx < self.board.size and 
                        0 <= ny < self.board.size and
                        self.board.grid[ny][nx] in [0, 1]):  # Only spread to trees/houses
                        
                        # Higher chance to spread to adjacent cells (60% chance)
                        if random.random() < 0.6:
                            new_fires.add((nx, ny))
        
        # Mark current fires as burned
        for (x, y) in self.burning_cells:
            self.board.grid[y][x] = 3
        
        # Ignite new cells
        self.burning_cells = new_fires
        for (x, y) in new_fires:
            self.board.grid[y][x] = 2
        
        self.time_step += 1
        return len(new_fires)
    
    def simulate(self, steps=50, initial_fires=3, delay=0.3):
        """Run complete fire simulation"""
        self.ignite_random(num_fires=initial_fires)
        
        for _ in range(steps):
            self.board.display()
            print(f"Time Step: {self.time_step} | Burning Cells: {len(self.burning_cells)}")
            
            new_fires = self.spread_fire()
            print(f"New fires started: {new_fires}")
            
            time.sleep(delay)
            
            if not self.burning_cells:
                print("\n🔥 Fire has burned out!")
                break

# Example usage
if __name__ == "__main__":
    # Create town with random house clusters (20-40% coverage)
    town = TownBoard(15)
    town.generate_town()
    
    # Simulate fire spread
    fire = FireSimulator(town)
    fire.simulate(steps=50, initial_fires=3, delay=0.2)

🌲 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵
🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵
🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🌲 🪵
🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵
🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵
🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵
🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵
🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵
🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵
🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵
🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵
🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵
🔥 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵
🔥 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵
🌲 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵 🪵

Legend: 🌲 Tree | 🏠 House | 🔥 Fire | 🪵 Burned
Time Step: 12 | Burning Cells: 2
New fires started: 0

🔥 Fire has burned out!
